In [ ]:
import os, sys

current_path = '/content/drive/My Drive/Colab Notebooks/300/'

os.chdir(current_path)

In [ ]:
# # pickle.format_version
# # print(pd.show_versions())
# !pip install pandas==0.25.3
!pip install mpl_finance

### **ARIMA**

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from datetime import datetime


def arima_test(close, use_rows=None):

  size = use_rows
  train, test = close[0:size].values, close[size:len(close)]
  test_shift = test.shift(1).values
  test = test.values
  # break

  history = list(train)
  predictions = list()
  err_ranges = list()
  for t in range(len(test)):
    
      if use_rows is not None:
        history = history[-use_rows:]
        
      model = ARIMA(history, order=(0, 2, 1))
      model_fit = model.fit(trend='c', disp=0)
      output = model_fit.forecast()
      # print(output)
      # break
      yhat = output[0]
      predictions.append(yhat)
      err_ranges.append(output[1])
      obs = test[t]
      # print('obs :', obs)
      history.append(obs)
      # break
      print('\r %.2f%%' % (t / len(test) * 100), end='')

  print(len(test), len(predictions))

  return predictions, err_ranges


# print(high)


def get_back_result(ohlcv, predictions, err_ranges, tp=0.04, sl=None, leverage=1, show_detail=False, show_plot=False, return_pr=False, cumsum=False, 
                    close_ver=False, reverse_short=False):

  
  # prev_close = ohlcv['close'].shift(1).values[-len(predictions):]
  high, low, test = np.split(ohlcv.values[-len(predictions):, [1, 2, 3]], 3, axis=1)

  if close_ver:
    predictions = ohlcv['close'].shift(1).values[-len(test):]

  fee = 0.0006
  long_profits = []
  short_profits = []
  liquidations = []
  win_cnt = 0
  for i in range(len(test)):

    # long_ep = predictions[i]
    # long_ep = prev_close[i]
    long_ep = (predictions[i] - err_ranges[i]) * (1 / (tp + 1))
    if sl is not None:
      long_sl = long_ep * (1 / (sl + 1))

    # assert long_ep < long_exit, 'long_exit < long_ep !, %s, %s' % (long_exit, long_ep)
    
    short_ep = (predictions[i] + err_ranges[i]) * (1 + tp)
    # short_ep = (predictions[i] + err_ranges[i]) * (1 / (1 - tp))
    if sl is not None:
      short_sl = short_ep * (1 / (1 - sl))

    # print((low[i]))

    #    long 우선   # <-- long & short 둘다 체결된 상황에서는 long 체결을 우선으로 한다.
    #    조건식 : 예상 종가가 이전 종가보다 상승하면, (bias 가 up 방향) 매수한다

    # if prev_close[i] < predictions[i]:

    if low[i] < long_ep:
      
      liquidation = low[i] / long_ep - fee
      l_liquidation = 1 + (liquidation - 1) * leverage
      liquidations.append(l_liquidation)

      if max(l_liquidation, 0) == 0:
        l_profit = 0
        # print('low[i], long_ep, l_liquidation :', low[i], long_ep, l_liquidation)
      else:

        if sl is not None:
          if low[i] < long_sl:
            profit = long_sl / long_ep - fee
          else:
            profit = test[i] / long_ep - fee

        else:
          profit = test[i] / long_ep - fee

        l_profit = 1 + (profit - 1) * leverage
        l_profit = max(l_profit, 0)
        
        if profit >= 1:
          win_cnt += 1

      long_profits.append(l_profit)
      short_profits.append(1.0)

      if show_detail:
        print("test[i], predictions[i], long_ep :", test[i], predictions[i], long_ep)

      # if high[i] > short_ep > low[i]: # 지정 대기가 아니라, 해당 price 가 지나면, long 한다.

      #   if not reverse_short:
      #     liquidation = short_ep / high[i]  - fee
      #   else:
      #     liquidation = low[i] / short_ep  - fee
      #   l_liquidation = 1 + (liquidation - 1) * leverage

      #   if max(l_liquidation, 0) == 0:
      #     l_profit = 0
      #   else:

      #     if sl is not None:
      #       if high[i] > short_sl:

      #         if not reverse_short:
      #           profit = short_ep / short_sl - fee
      #         else:
      #           profit = short_sl / short_ep - fee

      #       else:
      #         if not reverse_short:
      #           profit = short_ep / test[i] - fee
      #         else:
      #           profit = test[i] / short_ep - fee

      #     else:

      #       if not reverse_short:
      #         profit = short_ep / test[i] - fee
      #       else:
      #         profit = test[i] / short_ep - fee

      #     l_profit = 1 + (profit - 1) * leverage
      #     l_profit = max(l_profit, 0)

      #     if profit >= 1:
      #       win_cnt += 1

      #   short_profits.append(l_profit)
      #   long_profits.append(1.0)

      #   if show_detail:
      #     print(test[i], predictions[i], short_ep)
      
    else:
      long_profits.append(1.0)
      short_profits.append(1.0)
      liquidations.append(1.0)


  long_win_ratio = sum(np.array(long_profits) > 1.0) / sum(np.array(long_profits) != 1.0)
  short_win_ratio = sum(np.array(short_profits) > 1.0) / sum(np.array(short_profits) != 1.0)
  long_frequency = sum(np.array(long_profits) != 1.0) / len(test)
  short_frequency = sum(np.array(short_profits) != 1.0) / len(test)
  if not cumsum:
    long_accum_profit = np.array(long_profits).cumprod()
    short_accum_profit = np.array(short_profits).cumprod()
  else:
    long_accum_profit = (np.array(long_profits) - 1.0).cumsum()
    short_accum_profit = (np.array(short_profits) - 1.0).cumsum()

  # print(win_ratio)

  if show_plot:

    plt.figure(figsize=(10, 5))
    plt.suptitle('tp=%.4f, lvrg=%d' % (tp, leverage))

    plt.subplot(151)
    plt.plot(liquidations)
    plt.title('liquidations')

    plt.subplot(152)
    plt.plot(long_profits)
    plt.title('Win Ratio : %.2f %%\nrequency : %.2f %%' % (long_win_ratio * 100, long_frequency * 100), color='black')
    # plt.show()

    # print()
    plt.subplot(153)
    plt.plot(long_accum_profit)
    plt.title('Accum_profit : %.2f' % long_accum_profit[-1], color='black')

    plt.subplot(154)
    plt.plot(short_profits)
    plt.title('Win Ratio : %.2f %%\nrequency : %.2f %%' % (short_win_ratio * 100, short_frequency * 100), color='black')
    # plt.show()

    # print()
    plt.subplot(155)
    plt.plot(short_accum_profit)
    plt.title('Accum_profit : %.2f' % short_accum_profit[-1], color='black')
    plt.show()

  return [long_win_ratio, short_win_ratio], [long_frequency, short_frequency], [long_accum_profit[-1], short_accum_profit[-1]], [long_profits, short_profits]


# get_back_result(tp=0.04, leverage=1, show_plot=True)


### load data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

date = "2021-05-17"
# date = "2021-04-08"
interval = '30m'
date_path = './candlestick_concated/%s/%s/' % (interval, date)
file_list = os.listdir(date_path)

print((file_list))

['2021-05-17 BTCUSDT.xlsx', '2021-05-17 ETHUSDT.xlsx', '2021-05-17 BCHUSDT.xlsx', '2021-05-17 XRPUSDT.xlsx', '2021-05-17 EOSUSDT.xlsx', '2021-05-17 LTCUSDT.xlsx', '2021-05-17 ETCUSDT.xlsx', '2021-05-17 LINKUSDT.xlsx', '2021-05-17 XLMUSDT.xlsx', '2021-05-17 ADAUSDT.xlsx', '2021-05-17 XMRUSDT.xlsx', '2021-05-17 SXPUSDT.xlsx', '2021-05-17 KAVAUSDT.xlsx', '2021-05-17 BANDUSDT.xlsx', '2021-05-17 DASHUSDT.xlsx', '2021-05-17 ZECUSDT.xlsx', '2021-05-17 XTZUSDT.xlsx', '2021-05-17 BNBUSDT.xlsx', '2021-05-17 ATOMUSDT.xlsx', '2021-05-17 ONTUSDT.xlsx', '2021-05-17 IOTAUSDT.xlsx', '2021-05-17 BATUSDT.xlsx', '2021-05-17 NEOUSDT.xlsx', '2021-05-17 QTUMUSDT.xlsx', '2021-05-17 WAVESUSDT.xlsx', '2021-05-17 MKRUSDT.xlsx', '2021-05-17 SNXUSDT.xlsx', '2021-05-17 DOTUSDT.xlsx', '2021-05-17 THETAUSDT.xlsx', '2021-05-17 ALGOUSDT.xlsx', '2021-05-17 KNCUSDT.xlsx', '2021-05-17 ZRXUSDT.xlsx', '2021-05-17 COMPUSDT.xlsx', '2021-05-17 OMGUSDT.xlsx']


### Local box tp optimization

In [ ]:
import pickle
import time
import datetime
from tqdm.notebook import tqdm

# with open('./arima_result/arima_opt_profit_ls_only_long_result_%s.pickle' % interval, 'rb') as f:
#   load_dict = pickle.load(f)


start_stamp = 0
# start_stamp = datetime.timestamp(pd.to_datetime('2021-02-12'))
print("start_stamp :", start_stamp)

save_dict = {}
tp_list = np.arange(0, 0.03, 0.002)
leverage_list = np.arange(1, 6, 1)
safety_threshold = 0.
long_index = 0

leverage = 3

# tp_list = [selected_tp]

for file in file_list:

  # file = '2021-02-07 ETH.xlsx'
  print(file)
  if not file.endswith('xlsx'):
    continue

  # if '04-08' not in file:
  #   continue

  if 'eth'.upper() not in file:
    continue
  

  key = file

  ohlcv = pd.read_excel(date_path + file, index_col=0)
  print('len(ohlcv) :', len(ohlcv))

# for key in load_dict.keys():



### append ex / in - clusion

In [ ]:
tp = best_tp

ep = ohlcv['close'].shift(1) * (1 / (tp + 1))
lvrg = 3
fee = 0.0006


best_pr = np.where(ohlcv['low'] < ep, (ohlcv['close'] / ep - fee - 1) * lvrg + 1, 1)

wr = len(best_pr[best_pr > 1]) / len(best_pr[best_pr != 1])

plt.subplot(121)
plt.plot(np.cumprod(best_pr))
plt.title("wr : %.3f" % wr)


#     add constraints   #
ma = ohlcv['close'].rolling(200).mean()

#       ma    # --> it works
best_pr = np.where(((best_pr) != 1) & (ohlcv['close'].shift(1) > ma.shift(1)), best_pr, 1)
# best_pr = np.where(((best_pr) != 1) & (ohlcv['close'].shift(1) < ma.shift(1)), best_pr, 1)



#          plot       #
wr = len(best_pr[best_pr > 1]) / len(best_pr[best_pr != 1])

plt.subplot(122)
plt.plot(np.cumprod(best_pr))
plt.title("wr : %.3f" % wr)
plt.show()
plt.close()

### prev get_back_result

In [ ]:

  # print(key)

  # if len(list(load_dict[key].keys())) > 3:
  #   print(list(load_dict[key].keys()))
  #   continue

  #       tp analysis     #
  # ohlcv = load_dict[key]['ohlcv']
  # predictions = load_dict[key]['predictions']
  # err_ranges = load_dict[key]['err_ranges']

  # ohlcv = ohlcv.iloc[:-3000]

  #     test span   #
  # test_size = 3000
  # look_back_size = 3000

  # wr_list, fr_list, ap_list = [], [], []
  # pr_list = []

  # predictions = ohlcv['close'].shift(1).values
  # err_ranges = np.zeros_like(predictions)

  # for tp in tqdm(tp_list):


  # # try:
  #   result = get_back_result(ohlcv, predictions, err_ranges, tp=tp, leverage=leverage, show_plot=False, reverse_short=False, show_detail=False)

  #   # print(np.min(result[-1]))
  #   pr_list.append(result[3][0])
  #   # print('tp, result[2] :', tp, result[2])

  #   # if round(leverage) == 1:

  #   wr_list.append(result[0])
  #   fr_list.append(result[1])
  #   ap_list.append(result[2])


  # argmax_l, argmax_s = np.argmax(ap_list, axis=0)

  # best_tp = tp_list[:len(ap_list)][argmax_l]

  # print("best_tp :", best_tp)


  # best_pr = pr_list[argmax_l]

### exclusion

In [ ]:
import mpl_finance as mf

plt.plot((best_pr))
plt.show()

# plt.plot(np.cumprod(best_pr))
# plt.show()

print(len(ohlcv), len(best_pr))

best_pr = np.array(best_pr)
rapid_down = np.argwhere(best_pr < 0.85)
# print(rapid_down)

plot_size = 45

for i in rapid_down:

  if i > plot_size:

    i = i[0]

    ohlc = ohlcv.iloc[i + 1 - plot_size : i + 1, :4].values
    ma120 = ohlcv['close'].rolling(120).mean().values[i + 1 - plot_size : i + 1]

    fig = plt.figure(figsize=(10, 4))
    ax = fig.add_subplot(111)


    index = np.arange(len(ohlc))
    candle = np.hstack((np.reshape(index, (-1, 1)), ohlc))
    mf.candlestick_ohlc(ax, candle, width=0.5, colorup='r', colordown='b')

    plt.plot(ma120)

    plt.show()
    plt.close()


### inclusion

In [ ]:
import mpl_finance as mf

plt.plot((best_pr))
plt.show()

# plt.plot(np.cumprod(best_pr))
# plt.show()

print(len(ohlcv), len(best_pr))

best_pr = np.array(best_pr)
rapid_down = np.argwhere(best_pr > 1.1)
# print(rapid_down)

plot_size = 45

for i in rapid_down:

  if i > plot_size:

    i = i[0]

    ohlc = ohlcv.iloc[i + 1 - plot_size : i + 1, :4].values
    ma120 = ohlcv['close'].rolling(120).mean().values[i + 1 - plot_size : i + 1]

    fig = plt.figure(figsize=(10, 4))
    ax = fig.add_subplot(111)


    index = np.arange(len(ohlc))
    candle = np.hstack((np.reshape(index, (-1, 1)), ohlc))
    mf.candlestick_ohlc(ax, candle, width=0.5, colorup='r', colordown='b')

    plt.plot(ma120)

    plt.show()
    plt.close()


### **Keep Connection**

In [ ]:
while 1: 1 

### std survey

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
# print(type(result_dict))

with open('./arima_result/prev_close_%s_%s.pickle' % (interval, "2021-05-07"), 'rb') as f:
  profit_result_dict = pickle.load(f)

keys = profit_result_dict.keys()
print(keys)
# keys = ['2021-02-07 ETH.xlsx']
# break


  
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 2500)

pairs = list(profit_result_dict.keys())
result_df = pd.DataFrame(index=pairs)
profit = []
max_lv_profit = []
pr_fr_std = []
max_pr_fr_std = []

long_index = 0

for key in keys:  
  # print(np.max(profit_result_dict[key]['ap_list']))
  profit.append(np.max(profit_result_dict[key]['ap_list'][:, [long_index]]))
  # print(np.max(profit_result_dict[key]['leverage_ap_list']))
  max_lv_profit.append(np.max(profit_result_dict[key]['leverage_ap_list'][:, [long_index]]))

  pr_fr_std.append(profit_result_dict[key]["pr_fr_std"])

  max_pr_fr_std.append(profit_result_dict[key]["max_pr_fr_std"])


# result_df['index'] = pairs
result_df['profit']= profit
result_df['max_lv_profit']= max_lv_profit
result_df['pr_fr_std'] = pr_fr_std
result_df['max_pr_fr_std'] = max_pr_fr_std
# print(result_df)

#     sort by values    #
print(result_df.sort_values(by='profit', ascending=False))
print(result_df.sort_values(by=['max_lv_profit'], ascending=False))
print(result_df.sort_values(by=['pr_fr_std'], ascending=True))
print(result_df.sort_values(by=['max_pr_fr_std'], ascending=True))

#     extract candidates    #
candis = result_df.sort_values(by=['max_lv_profit'], ascending=False).index
print(candis)



#         save dict 2       #
# with open('./arima_result/prev_close_candi_profit_ls_only_long_result_%s.pickle' % interval, 'wb') as f:
#   pickle.dump(profit_result_dict, f)


dict_keys(['2021-05-07 OMGUSDT.xlsx', '2021-05-07 KNCUSDT.xlsx', '2021-05-07 COMPUSDT.xlsx', '2021-05-07 LTCUSDT.xlsx', '2021-05-07 LINKUSDT.xlsx', '2021-05-07 ETHUSDT.xlsx', '2021-05-07 XRPUSDT.xlsx', '2021-05-07 EOSUSDT.xlsx', '2021-05-07 ETCUSDT.xlsx', '2021-05-07 BTCUSDT.xlsx', '2021-05-07 ADAUSDT.xlsx', '2021-05-07 BCHUSDT.xlsx', '2021-05-07 SXPUSDT.xlsx', '2021-05-07 BNBUSDT.xlsx', '2021-05-07 ONTUSDT.xlsx', '2021-05-07 XMRUSDT.xlsx', '2021-05-07 XLMUSDT.xlsx', '2021-05-07 DASHUSDT.xlsx', '2021-05-07 ZECUSDT.xlsx', '2021-05-07 ATOMUSDT.xlsx', '2021-05-07 BANDUSDT.xlsx', '2021-05-07 KAVAUSDT.xlsx', '2021-05-07 XTZUSDT.xlsx', '2021-05-07 BATUSDT.xlsx', '2021-05-07 IOTAUSDT.xlsx', '2021-05-07 THETAUSDT.xlsx', '2021-05-07 DOTUSDT.xlsx', '2021-05-07 ZRXUSDT.xlsx', '2021-05-07 MKRUSDT.xlsx', '2021-05-07 NEOUSDT.xlsx', '2021-05-07 QTUMUSDT.xlsx', '2021-05-07 ALGOUSDT.xlsx', '2021-05-07 WAVESUSDT.xlsx', '2021-05-07 SNXUSDT.xlsx'])
                                         profit         m

### **Candi Result in sorted way**

In [ ]:
long_index = 0
safety_threshold = 0.6



for i in range(len(candis)):

  keys = [candis[i]]

  # if '04-08' not in candis[i]:
  #   continue

  # if 'eth'.upper() not in candis[i]:
  #   continue

  plt.figure(figsize=(35, 7))
  plt.suptitle('%s %s' % (interval, keys))

  plt.subplot(1,10,1)
  plt.title('win ratio')  
  for key in keys:  
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['wr_list'][:, [long_index]])


  plt.subplot(1,10,2)
  plt.title('frequency')  
  for key in keys:  
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['fr_list'][:, [long_index]])

    
  plt.subplot(1,10,3)
  for key in keys:  
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['ap_list'][:, [long_index]])
    argmax = np.argmax(profit_result_dict[key]['ap_list'][:, [long_index]])
    peak_tp = profit_result_dict[key]['tp_list'][argmax]
    plt.axvline(peak_tp, linestyle='--')
    plt.title('acc profit, max at %.4f' % (peak_tp))  

  plt.subplot(1,10,4)
  plt.title('max acc profit by leverage')  
  for key in keys:  
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['max_ap_list'][:, [long_index]], label=key)
    argmax = np.argmax(profit_result_dict[key]['max_ap_list'][:, [long_index]])
    max_peak_tp = profit_result_dict[key]['tp_list'][argmax]
    plt.axvline(max_peak_tp, linestyle='--')
    plt.title('max acc profit, max at %.4f' % (max_peak_tp))  


  for key in keys:  
    # print(profit_result_dict[key]['leverage_ap_list'])

    for tp in [peak_tp, max_peak_tp]:

      if tp == peak_tp:
        plt.subplot(1,10,5)
      else:
        plt.subplot(1,10,6)

      #     leverage analysis     #
      ohlcv = pd.read_excel(date_path + key, index_col=0)

      # ohlcv = profit_result_dict[key]['ohlcv']
      # predictions = profit_result_dict[key]['predictions']
      # err_ranges = profit_result_dict[key]['err_ranges']
      
      predictions = ohlcv['close'].shift(1).values[-3000:]
      err_ranges = np.zeros_like(predictions)[-3000:]

      print("len(predictions) :", len(predictions))
      
      leverage_list = profit_result_dict[key]['leverage_list']
      # leverage_list = [4]
      temp_ap_list = list()
      temp_pr_list = list()

      for leverage in leverage_list:

        try:
          # result = get_back_result(ohlcv, predictions, err_ranges, tp=0.018, leverage=leverage, reverse_short=False)
          result = get_back_result(ohlcv, predictions, err_ranges, tp=tp, show_plot=False, leverage=leverage, reverse_short=False)

          # if min(result[-1][long_index]) < safety_threshold:
          #   continue

          # print("leverage, result[2] :", leverage, result[2])

          temp_ap_list.append(result[2][long_index])
          temp_pr_list.append(result[3][long_index])

          # if round(leverage) == 1:
          #   temp_pr_list = result[3]

        except Exception as e:
          print(e)
          break

      
      # profit_result_dict[key]['pr_list'] = temp_pr_list

      plt.plot(profit_result_dict[key]['leverage_list'][:len(temp_ap_list)], temp_ap_list, label=key)
      argmax = np.argmax(temp_ap_list)
      plt.title('acc profit by leverage\n max at tp=%.4f lvrg=%.0f' % (tp, profit_result_dict[key]['leverage_list'][:len(temp_ap_list)][argmax]))  
      plt.axvline(profit_result_dict[key]['leverage_list'][:len(temp_ap_list)][argmax], linestyle='--')

      if tp == peak_tp:
        plt.subplot(1,10,7)
      else:
        plt.subplot(1,10,9)

      plt.plot(temp_pr_list[argmax])
    # plt.xlim(0, 13)

    
  # plt.subplot(1,10,7)
  for key in keys:  
    pr_list = profit_result_dict[key]['pr_list']
    # plt.plot(pr_list)

    pr_fr_std = profit_result_dict[key]['pr_fr_std']
    pr_list_fr = list(map(lambda x : 0 if x == 1.0 else 1, pr_list))

    plt.subplot(1,10,8)
    plt.title('std : %s' % pr_fr_std)
    plt.plot(pr_list_fr)

  # plt.subplot(1,10,9)
  for key in keys:  
    pr_list = profit_result_dict[key]['max_pr_list']
    # plt.plot(pr_list)

    pr_fr_std = profit_result_dict[key]['max_pr_fr_std']
    pr_list_fr = list(map(lambda x : 0 if x == 1.0 else 1, pr_list))

    plt.subplot(1,10,10)
    plt.title('std : %s' % pr_fr_std)
    plt.plot(pr_list_fr)


  
  

  plt.show()

  

Output hidden; open in https://colab.research.google.com to view.

#### **RM - Maximum Loss**

In [ ]:
leverage_list2 = np.arange(1, 6, 1)

for i in range(len(candis)):

  keys = [candis[i]]
  
  # if 'eth'.upper() not in candis[i]:
  #   continue

  # plt.figure(figsize=(35, 10))
  # plt.suptitle('%s %s' % (interval, keys))

  # plt.subplot(1,10,3)
  for key in keys:  
    # plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['ap_list'])
    argmax = np.argmax(profit_result_dict[key]['ap_list'][:, [long_index]])
    peak_tp = profit_result_dict[key]['tp_list'][argmax]
    # plt.axvline(peak_tp, linestyle='--')
    # plt.title('acc profit, max at %.4f' % (peak_tp))  

  # plt.subplot(1,10,4)
  # plt.title('max acc profit by leverage')  
  for key in keys:  
    # plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['max_ap_list'], label=key)
    argmax = np.argmax(profit_result_dict[key]['max_ap_list'][:, [long_index]])
    max_peak_tp = profit_result_dict[key]['tp_list'][argmax]
    # plt.axvline(max_peak_tp, linestyle='--')
    # plt.title('max acc profit, max at %.4f' % (max_peak_tp))  


  for key in keys:  
    # print(profit_result_dict[key]['leverage_ap_list'])

    for tp in [peak_tp, max_peak_tp]:

      # if tp == peak_tp:
      #   plt.subplot(1,10,5)
      # else:
      #   plt.subplot(1,10,6)

      #     leverage analysis     #
      ohlcv = profit_result_dict[key]['ohlcv']
      predictions = profit_result_dict[key]['predictions']
      err_ranges = profit_result_dict[key]['err_ranges']
      # leverage_list = profit_result_dict[key]['leverage_list']
      # temp_ap_list = list()
      # temp_pr_list = list()

      for leverage in leverage_list2:

        try:
          print('-------------- %s --------------' % key)
          result = get_back_result(ohlcv, predictions, err_ranges, tp=tp, leverage=leverage, show_plot=True, reverse_short=False, show_detail=False)
          # temp_ap_list.append(result[2])
          # temp_pr_list.append(result[3])

          # if round(leverage) == 1:
          #   temp_pr_list = result[3]

        except Exception as e:
          print(e)
          break
        
  # break

  

Output hidden; open in https://colab.research.google.com to view.